# Openning Data

In [1]:
using SparseArrays
using JuMP
#using GLPK
using Ipopt
using LinearAlgebra
using MAT

In [2]:
file = matopen("R4T2.mat")
varnames = keys(file)

4-element Array{String,1}:
 "L"
 "S"
 "U"
 "lambda"

In [3]:
S = read(file, "S")
L = read(file, "L")
U = read(file, "U")
lambda = read(file,"lambda")
using DelimitedFiles
#answer= readdlm("Answers/output-41.txt",',',Float64);

In [4]:
(m,n)=size(S)
(r,c) = size(L)

(13094, 50)

# constructing model

In [10]:
function opt_vector0(S,l,u,z)
    model = Model(Ipopt.Optimizer)
    @variable(model, l[i] <= v[i = 1:r] <= u[i])
    @variable(model,a[1:r])
    @constraint(model, S*v .== 0)
    @constraint(model, a .>= v)
    @constraint(model, a .>= -v)
    @NLobjective(model,Min,sum(sqrt(1+z[i]+a[i]) for i=1:r))
    optimize!(model)
    return JuMP.value.(v)
end

opt_vector0 (generic function with 1 method)

In [5]:
function opt_vector(S,l,u,z,lambda)
    model = Model(Ipopt.Optimizer)
    @variable(model, l[i] <= v[i = 1:r] <= u[i])
    @variable(model,a[1:r])
    @constraint(model, S*v .==  0)
    @constraint(model, a .>= v)
    @constraint(model, a .>= -v)
    @NLobjective(model,Min,sum(log(1+z[i]+a[i]) for i=1:r) + lambda*sum(log(1+(S*v)[j]) for j=1:m))
    optimize!(model)
    return JuMP.value.(v)
end

opt_vector (generic function with 1 method)

In [6]:
function zero_rows(V)
    k=0
    for i in 1:size(V)[1]
        if sum(V[i,:].*V[i,:])<1e-10
            k+=1
        end
    end
    return k
end
function not_zero_rows(V)
    indexes= zeros(Int64,size(V)[1],1)
    k=1
    for i in 1:size(V)[1]
        if sum(V[i,:].*V[i,:])>=1e-10
            indexes[k]= i
            k+=1
        end
    end
    return indexes[1:k-1]
end

not_zero_rows (generic function with 1 method)

In [7]:
function row_squares(V)
    r,c = size(V,1), size(V,2)
    squares = zeros(r,1)
    for i in 1:r
        z=0
        for j in 1:c
           z += (V[i,j])^2
        end
        #squares[i]=sqrt(z)
        squares[i]=z
    end
    return squares
end

row_squares (generic function with 1 method)

In [52]:
"""
non_zero_indexes= not_zero_rows(answer)
S= S[1:size(S)[1],non_zero_indexes]
U= U[non_zero_indexes,1:size(U)[2]]
L= L[non_zero_indexes,1:size(L)[2]]
m= size(S)[1]
r= size(U)[1]
c= size(U)[2]
"""

50

In [16]:
l = L[:,1]
u = U[:,1]
z = ones(r,1)
v = opt_vector0(S,l,u,z)
#zero_rows(v)

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     5663
Number of nonzeros in inequality constraint Jacobian.:    28728
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    14364
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1270
                     variables with only upper bounds:        0
Total number of equality constraints.................:     8404
Total number of inequality constraints...............:    26188
        inequality constraints with only lower bounds:    26188
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

13094-element Array{Float64,1}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 -6.961629041525816e-17
  4.6202617811818665e-11
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [17]:
V=v;

In [18]:
for i in 2:c
    if isdefined(Main, :IJulia)
        Main.IJulia.stdio_bytes[] = 0
    end
    l = L[:,i]
    u = U[:,i]
    z = row_squares(V)
    v = opt_vector0(S,l,u,z)
    V = [V v]
    print("**i : ",i,"\n")
    #print("***",zero_rows(V),"\n") 
end

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     6275
Number of nonzeros in inequality constraint Jacobian.:    28854
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    14427
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1333
                     variables with only upper bounds:        0
Total number of equality constraints.................:     8404
Total number of inequality constraints...............:    26188
        inequality constraints with only lower bounds:    26188
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

**i : 2
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     5263
Number of nonzeros in inequality constraint Jacobian.:    28598
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    14299
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1205
                     variables with only upper bounds:        0
Total number of equality constraints.................:     8404
Total number of inequality constraints...............:    26188
        inequality constraints with only lower bounds:    26188
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  l

**i : 3
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     5411
Number of nonzeros in inequality constraint Jacobian.:    28614
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    14307
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1213
                     variables with only upper bounds:        0
Total number of equality constraints.................:     8404
Total number of inequality constraints...............:    26188
        inequality constraints with only lower bounds:    26188
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  l

**i : 4
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     5703
Number of nonzeros in inequality constraint Jacobian.:    28774
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    14387
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1293
                     variables with only upper bounds:        0
Total number of equality constraints.................:     8404
Total number of inequality constraints...............:    26188
        inequality constraints with only lower bounds:    26188
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  l

**i : 5
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     5831
Number of nonzeros in inequality constraint Jacobian.:    28850
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    14425
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1331
                     variables with only upper bounds:        0
Total number of equality constraints.................:     8404
Total number of inequality constraints...............:    26188
        inequality constraints with only lower bounds:    26188
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  l

...

**i : 46
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     5500
Number of nonzeros in inequality constraint Jacobian.:    28702
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    14351
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1257
                     variables with only upper bounds:        0
Total number of equality constraints.................:     8404
Total number of inequality constraints...............:    26188
        inequality constraints with only lower bounds:    26188
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  

**i : 47
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     5062
Number of nonzeros in inequality constraint Jacobian.:    28568
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    14284
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1190
                     variables with only upper bounds:        0
Total number of equality constraints.................:     8404
Total number of inequality constraints...............:    26188
        inequality constraints with only lower bounds:    26188
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  

**i : 48
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     5828
Number of nonzeros in inequality constraint Jacobian.:    28770
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    14385
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1291
                     variables with only upper bounds:        0
Total number of equality constraints.................:     8404
Total number of inequality constraints...............:    26188
        inequality constraints with only lower bounds:    26188
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  

**i : 49
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     5470
Number of nonzeros in inequality constraint Jacobian.:    28732
Number of nonzeros in Lagrangian Hessian.............:    13094

Total number of variables............................:    14366
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1272
                     variables with only upper bounds:        0
Total number of equality constraints.................:     8404
Total number of inequality constraints...............:    26188
        inequality constraints with only lower bounds:    26188
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  

**i : 50


In [19]:
n1= zero_rows(V)
n2= zero_rows((S*V)')
print(n1,' ',n2,' ',n1+lambda*n2)

12535 50 32176.0

In [59]:
answer[non_zero_indexes,:].= V[:,:];

# writing to output file

In [22]:
using DelimitedFiles
open("output.txt","w") do io
    writedlm(io, V,",")
end